In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 6.6, 7.8),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 34, 38),
                'subsample': trial.suggest_uniform('subsample', 0.825, 0.875),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.645, 0.695),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 5.5e-2, 7.5e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 23, 27),
              'num_leaves':trial.suggest_int('num_leaves', 19, 29),
              'max_depth':trial.suggest_int('max_depth', 2, 3),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 732, 762),
              'cat_smooth':trial.suggest_int('cat_smooth', 88, 92),
              'cat_l2':trial.suggest_loguniform('cat_l2', 3.7, 6.25)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-21 02:36:49,215] A new study created in memory with name: no-name-128cd841-23a8-41c6-bab0-2a69f026ebdc


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717089
[2000]	valid_0's rmse: 0.715067
[3000]	valid_0's rmse: 0.714583
Early stopping, best iteration is:
[3090]	valid_0's rmse: 0.714566
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717119
[2000]	valid_0's rmse: 0.715086
[3000]	valid_0's rmse: 0.714727
Early stopping, best iteration is:
[3388]	valid_0's rmse: 0.714675


[I 2021-08-21 02:38:08,318] Trial 0 finished with value: 0.7144695779946586 and parameters: {'reg_lambda': 7.685780275889322, 'reg_alpha': 35.06254186395001, 'subsample': 0.8673627928188785, 'colsample_bytree': 0.6699163816690578, 'learning_rate': 0.0683907765278896, 'min_child_samples': 24, 'num_leaves': 21, 'max_depth': 3, 'max_bin': 738, 'cat_smooth': 90, 'cat_l2': 5.569087556194927}. Best is trial 0 with value: 0.7144695779946586.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.720394
[2000]	valid_0's rmse: 0.716788
[3000]	valid_0's rmse: 0.715269
[4000]	valid_0's rmse: 0.714592
[5000]	valid_0's rmse: 0.714362
[6000]	valid_0's rmse: 0.714273
Early stopping, best iteration is:
[5816]	valid_0's rmse: 0.714261
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.720472
[2000]	valid_0's rmse: 0.716873
[3000]	valid_0's rmse: 0.715281
[4000]	valid_0's rmse: 0.714641
[5000]	valid_0's rmse: 0.714371
[6000]	valid_0's rmse: 0.714259
Early stopping, best iteration is:
[5988]	valid_0's rmse: 0.714258


[I 2021-08-21 02:40:00,782] Trial 1 finished with value: 0.7141631430900747 and parameters: {'reg_lambda': 6.954468373460137, 'reg_alpha': 35.06911897318049, 'subsample': 0.8592515353056105, 'colsample_bytree': 0.6566066931274556, 'learning_rate': 0.07434827317454108, 'min_child_samples': 25, 'num_leaves': 24, 'max_depth': 2, 'max_bin': 760, 'cat_smooth': 92, 'cat_l2': 5.008394512233755}. Best is trial 1 with value: 0.7141631430900747.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.720993
[2000]	valid_0's rmse: 0.717265
[3000]	valid_0's rmse: 0.715578
[4000]	valid_0's rmse: 0.714721
[5000]	valid_0's rmse: 0.714309
[6000]	valid_0's rmse: 0.714101
Early stopping, best iteration is:
[6587]	valid_0's rmse: 0.714031
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.721187
[2000]	valid_0's rmse: 0.717505
[3000]	valid_0's rmse: 0.715776
[4000]	valid_0's rmse: 0.714898
[5000]	valid_0's rmse: 0.714492
[6000]	valid_0's rmse: 0.714351
[7000]	valid_0's rmse: 0.714292
Early stopping, best iteration is:
[6811]	valid_0's rmse: 0.714289


[I 2021-08-21 03:35:18,087] Trial 28 finished with value: 0.7140684064247501 and parameters: {'reg_lambda': 6.855126036474554, 'reg_alpha': 35.62485319025951, 'subsample': 0.8746495151598167, 'colsample_bytree': 0.6527820812378021, 'learning_rate': 0.0652054076536791, 'min_child_samples': 25, 'num_leaves': 23, 'max_depth': 2, 'max_bin': 749, 'cat_smooth': 88, 'cat_l2': 5.438983341097058}. Best is trial 22 with value: 0.7139793127526392.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717088
[2000]	valid_0's rmse: 0.715113
[3000]	valid_0's rmse: 0.714682
Early stopping, best iteration is:
[3173]	valid_0's rmse: 0.71465
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717044
[2000]	valid_0's rmse: 0.714959
[3000]	valid_0's rmse: 0.714587
Early stopping, best iteration is:
[3214]	valid_0's rmse: 0.714522


[I 2021-08-21 03:36:36,290] Trial 29 finished with value: 0.7144388541484309 and parameters: {'reg_lambda': 7.785765097700414, 'reg_alpha': 36.33867180724996, 'subsample': 0.8661534137571989, 'colsample_bytree': 0.6457114939654622, 'learning_rate': 0.06898861926858153, 'min_child_samples': 24, 'num_leaves': 21, 'max_depth': 3, 'max_bin': 739, 'cat_smooth': 90, 'cat_l2': 5.746910227747635}. Best is trial 22 with value: 0.7139793127526392.


Number of finished trials: 30
Best trial: {'reg_lambda': 6.609931804562087, 'reg_alpha': 35.89337620630626, 'subsample': 0.8737357601914469, 'colsample_bytree': 0.6612958156067924, 'learning_rate': 0.06767790040126322, 'min_child_samples': 25, 'num_leaves': 23, 'max_depth': 2, 'max_bin': 747, 'cat_smooth': 89, 'cat_l2': 5.885433554272715}


In [13]:
study.best_params

{'reg_lambda': 6.609931804562087,
 'reg_alpha': 35.89337620630626,
 'subsample': 0.8737357601914469,
 'colsample_bytree': 0.6612958156067924,
 'learning_rate': 0.06767790040126322,
 'min_child_samples': 25,
 'num_leaves': 23,
 'max_depth': 2,
 'max_bin': 747,
 'cat_smooth': 89,
 'cat_l2': 5.885433554272715}

# Log

====== Ordinal encoding =========

0.7179915711010899 row-wise noise ver4

0.7180313553153085 random noise ver5

0.7145452167048295 no noise ver6

0.7143969338945801 no noise ver10 (narrow space)

0.7141318460599843 no noise ver11 (narrow space)

0.7140261333995335 no noise ver12 (narrow space)

====== One-hot encoding =========

0.7242254028037789 no noise ver7

0.7264762744397595 random noise ver8

0.7265375795882082 row-wise noise ver9


